In [4]:
import torch
from pathlib import Path
import numpy as np

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
)
import joblib

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

PROJECT_ROOT = Path(r"D:\do-an-tot-nghiep")  

# 1) Model tóm tắt vit5 đã fine-tune
SUM_MODEL_DIR = PROJECT_ROOT / "models" / "best_model_combined"

# 2) Model PhoBERT phân loại đã fine-tune
CLS_MODEL_DIR = PROJECT_ROOT / "models" / "phobert"

# 3) Label encoder lưu lúc train PhoBERT
LBL_ENCODER_PATH = PROJECT_ROOT / "models" / "phobert" / "label_encoder.pkl"

print("SUM_MODEL_DIR exists:", SUM_MODEL_DIR.exists())
print("CLS_MODEL_DIR exists:", CLS_MODEL_DIR.exists())
print("LBL_ENCODER_PATH exists:", LBL_ENCODER_PATH.exists())


PyTorch version: 2.6.0+cu124
CUDA available: True
Using device: cuda
SUM_MODEL_DIR exists: True
CLS_MODEL_DIR exists: True
LBL_ENCODER_PATH exists: True


load model tom tat

In [5]:
assert SUM_MODEL_DIR.exists(), "SUM_MODEL_DIR không tồn tại, kiểm tra lại đường dẫn!"

sum_tokenizer = AutoTokenizer.from_pretrained(SUM_MODEL_DIR)
sum_model = AutoModelForSeq2SeqLM.from_pretrained(SUM_MODEL_DIR)
sum_model.to(device)
sum_model.eval()

print("Loaded summarization model from:", SUM_MODEL_DIR)
print("Summarization model device:", next(sum_model.parameters()).device)

MAX_SOURCE_LEN = 1500   
MAX_NEW_TOKENS = 220    # ~200 từ tóm tắt


Loaded summarization model from: D:\do-an-tot-nghiep\models\best_model_combined
Summarization model device: cuda:0


Hàm tóm tắt

In [6]:
@torch.no_grad()
def summarize_raw(text: str,
                  max_new_tokens: int = MAX_NEW_TOKENS,
                  num_beams: int = 5) -> str:
    """
    Tóm tắt 1 đoạn text bất kỳ (đã tự nối title + body trước đó).
    """
    text = str(text).strip()
    if not text:
        return ""

    inputs = sum_tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=MAX_SOURCE_LEN,
    ).to(device)

    if device.type == "cuda":
        with torch.amp.autocast("cuda"):
            output_ids = sum_model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                num_beams=num_beams,
                length_penalty=0.8,
                no_repeat_ngram_size=3,
                repetition_penalty=1.05,
            )
    else:
        output_ids = sum_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=num_beams,
            length_penalty=0.8,
            no_repeat_ngram_size=3,
            repetition_penalty=1.05,
        )

    summary = sum_tokenizer.decode(
        output_ids[0],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True,
    )
    return summary.strip()


def summarize_news(title: str, body: str) -> str:
    """
    Tạo input_text giống khi train: title + ". " + body, rồi tóm tắt.
    """
    input_text = (str(title).strip() + ". " + str(body).strip()).strip()
    return summarize_raw(input_text)


Load PhoBERT + label encoder

In [7]:
assert CLS_MODEL_DIR.exists(), "CLS_MODEL_DIR không tồn tại"
assert LBL_ENCODER_PATH.exists(), "LBL_ENCODER_PATH không tồn tại"

# Tokenizer PhoBERT (có thể load từ vinai/phobert-base hoặc từ CLS_MODEL_DIR đều được)
PHOBERT_NAME = "vinai/phobert-base"

cls_tokenizer = AutoTokenizer.from_pretrained(PHOBERT_NAME, use_fast=False)
cls_model = AutoModelForSequenceClassification.from_pretrained(CLS_MODEL_DIR)
cls_model.to(device)
cls_model.eval()

label_encoder = joblib.load(LBL_ENCODER_PATH)
label_classes = label_encoder.classes_
num_labels = len(label_classes)

print("Loaded PhoBERT classifier from:", CLS_MODEL_DIR)
print("Số lớp:", num_labels)
print("Label classes:", list(label_classes))

# Chiều dài input cho PhoBERT (giống lúc train)
CLS_MAX_LENGTH = 256  


Loaded PhoBERT classifier from: D:\do-an-tot-nghiep\models\phobert
Số lớp: 11
Label classes: ['Chính trị', 'Du lịch', 'Giáo dục', 'Giải trí', 'Khoa học công nghệ', 'Kinh doanh', 'Pháp luật', 'Sức khỏe', 'Thế giới', 'Thể thao', 'Đời sống']


Hàm phân loại + % nhãn

In [8]:
@torch.no_grad()
def classify_summary(summary_text: str, top_k: int = 3):
    """
    Nhận 1 summary, trả về top_k nhãn có xác suất cao nhất.
    Mỗi phần tử: {"label": ..., "prob": 0-1, "percent": 0-100}
    """
    summary_text = str(summary_text).strip()
    if not summary_text:
        return []

    enc = cls_tokenizer(
        summary_text,
        return_tensors="pt",
        truncation=True,
        max_length=CLS_MAX_LENGTH,
    ).to(device)

    outputs = cls_model(**enc)
    logits = outputs.logits[0]          # (num_labels,)
    probs = torch.softmax(logits, dim=-1).cpu().numpy()

    idx_sorted = np.argsort(probs)[::-1]  # sort giảm dần

    results = []
    for i in idx_sorted[:top_k]:
        results.append({
            "label": label_classes[i],
            "prob": float(probs[i]),
            "percent": float(probs[i] * 100),
        })
    return results


def classify_from_summary(summary_text: str, top_k: int = 3):
    """
    In ra kết quả phân loại đẹp hơn.
    """
    res = classify_summary(summary_text, top_k=top_k)
    if not res:
        print("Summary rỗng, không phân loại được.")
        return

    print("== PHÂN LOẠI (trên summary) ==")
    for r in res:
        print(f"- {r['label']}: {r['percent']:.2f}%")

    main_label = res[0]["label"]
    print(f"\nNhãn chính: {main_label} ({res[0]['percent']:.2f}%)")


Pipeline end-to-end: title + body → summary + % nhãn

In [9]:
def demo_news(title: str, body: str, top_k: int = 3):
    print("INPUT TITLE")
    print(title)
    print("\nINPUT BODY (rút gọn)")
    print(body[:800], "...\n")

    # 1) Tóm tắt
    summary = summarize_news(title, body)
    print("SUMMARY")
    print(summary, "\n")

    # 2) Phân loại trên summary
    res = classify_summary(summary, top_k=top_k)
    if not res:
        print("Không phân loại được.")
        return

    print("PHÂN LOẠI (trên summary)")
    for r in res:
        print(f"- {r['label']}: {r['percent']:.2f}%")

    print(f"\nNhãn chính: {res[0]['label']} ({res[0]['percent']:.2f}%)")


# Ví dụ: dán 1 bài báo vào đây để test
sample_title = "Trung Quốc xây đảo nhân tạo nổi chịu được vụ nổ hạt nhân"
sample_body = """
Trung Quốc đang xây cơ sở nghiên cứu khoa học khổng lồ nặng 78.000 tấn, có thiết kế hai thân bán chìm, dự kiến hoạt động năm 2028.

Công trình mang tên "Cơ sở nghiên cứu lưu trú nổi mọi thời tiết biển sâu", do nhóm chuyên gia từ Đại học Giao thông Thượng Hải (SJTU) và Tập đoàn Đóng tàu Nhà nước Trung Quốc (CSSC) thực hiện.

Theo SCMP, đây là đảo nhân tạo nổi đầu tiên trên thế giới có boongke chống hạt nhân. Nó có thể chuyển đổi chấn động từ vụ nổ hạt nhân thành lực ép nhẹ nhờ thiết kế chống nổ độc đáo, sử dụng các tấm siêu vật liệu gồm nhiều lớp.

Đảo có sức chứa 238 người và hoạt động được 4 tháng mà không cần tiếp tế. Công trình kết hợp giữa tính di động của tàu thuyền và sự dồi dào về nhiên liệu, vật tư của các trạm nghiên cứu. Nó có thể di chuyển với tốc độ 28 km/h, cho phép các nhà nghiên cứu quan sát vùng biển sâu dài ngày, thử nghiệm thiết bị hàng hải thế hệ mới và tìm hiểu những công nghệ khai thác khoáng sản đáy biển.

Cơ sở nghiên cứu lưu trú nổi mọi thời tiết biển sâu được thiết kế bán chìm, tức phần lớn nằm dưới mực nước biển, giúp giữ ổn định tốt ngay cả khi biển động. Chỉ tầng trên và các module phòng thí nghiệm vẫn nằm phía trên mặt nước. Công trình dài 85 m, rộng 38 m, được cho là có thể chống chọi với siêu bão.

"Cơ sở khoa học lớn dưới biển sâu này được thiết kế để lưu trú dài ngày trong mọi điều kiện thời tiết", nhóm chuyên gia, dẫn đầu bởi giáo sư Yang Deqing tại SJTU, viết trong nghiên cứu đăng trên tạp chí Chinese Journal of Ship Research tháng này. "Cấu trúc phía trên chứa các khoang trọng yếu giúp đảm bảo nguồn điện khẩn cấp, thông tin liên lạc và kiểm soát dẫn đường. Do đó, khả năng chống chịu với vụ nổ hạt nhân của cấu trúc vô cùng thiết yếu".

Công trình mới có thể trang bị hệ thống cảm biến biển sâu, thiết bị giám sát môi trường và phương tiện lặn tự động. Nó cũng được kỳ vọng giúp giải quyết một số rủi ro như những vụ nổ dưới nước, tai nạn hoặc tác động của thời tiết khắc nghiệt.
"""

demo_news(sample_title, sample_body, top_k=5)


INPUT TITLE
Trung Quốc xây đảo nhân tạo nổi chịu được vụ nổ hạt nhân

INPUT BODY (rút gọn)

Trung Quốc đang xây cơ sở nghiên cứu khoa học khổng lồ nặng 78.000 tấn, có thiết kế hai thân bán chìm, dự kiến hoạt động năm 2028.

Công trình mang tên "Cơ sở nghiên cứu lưu trú nổi mọi thời tiết biển sâu", do nhóm chuyên gia từ Đại học Giao thông Thượng Hải (SJTU) và Tập đoàn Đóng tàu Nhà nước Trung Quốc (CSSC) thực hiện.

Theo SCMP, đây là đảo nhân tạo nổi đầu tiên trên thế giới có boongke chống hạt nhân. Nó có thể chuyển đổi chấn động từ vụ nổ hạt nhân thành lực ép nhẹ nhờ thiết kế chống nổ độc đáo, sử dụng các tấm siêu vật liệu gồm nhiều lớp.

Đảo có sức chứa 238 người và hoạt động được 4 tháng mà không cần tiếp tế. Công trình kết hợp giữa tính di động của tàu thuyền và sự dồi dào về nhiên liệu, vật tư của các trạm nghiên cứu. Nó có thể di chuyển với tốc độ 28 km/h, cho phép các nhà n ...

SUMMARY
Trung Quốc đang xây cơ sở nghiên cứu khoa học khổng lồ nặng 78.000 tấn, có thiết kế hai thân bá